# Segmenting and Clustering the neighbourhoods in Toronto

In [28]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

import json # library to handle JSON files

import requests # library to handle requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors # Matplotlib and associated plotting modules

from sklearn.cluster import KMeans # import k-means from clustering stage

!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup # website scraping libraries and packages in Python from BeautifulSoup 

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim  # convert an address into latitude and longitude values

print("Libraries imported.")

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [29]:
import folium # map rendering library

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [30]:
# GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [31]:
soup = BeautifulSoup(data, 'html.parser')

In [32]:
postalCodeList = []
boroughList = []
neighborhoodList = []

In [33]:
soup.find('table').find_all('tr')

# find all the rows of the table
soup.find('table').find_all('tr')

# for each row of the table, find all the table data
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')

In [34]:
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n'))

In [35]:

toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned
1,M2A\n,Not assigned\n,Not assigned
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"


In [36]:
toronto_df_drop = toronto_df[toronto_df.Neighborhood != "Not assigned"].reset_index(drop=True)
toronto_df_drop.head()                                                 

,PostalCode,Borough,Neighborhood
0,M3A\n,North York\n,Parkwoods
1,M4A\n,North York\n,Victoria Village
2,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"
3,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights"
4,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government"


In [37]:
toronto_df_grouped = toronto_df_drop.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B\n,Scarborough\n,"Malvern, Rouge"
1,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
2,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
3,M1G\n,Scarborough\n,Woburn
4,M1H\n,Scarborough\n,Cedarbrae


In [38]:
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped

,PostalCode,Borough,Neighborhood
0,M1B\n,Scarborough\n,"Malvern, Rouge"
1,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
2,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
3,M1G\n,Scarborough\n,Woburn
4,M1H\n,Scarborough\n,Cedarbrae
...,...,...,...
98,M9N\n,York\n,Weston
99,M9P\n,Etobicoke\n,Westmount
100,M9R\n,Etobicoke\n,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V\n,Etobicoke\n,"South Steeles, Silverstone, Humbergate, Jamest..."


In [39]:

# create a new test dataframe
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_grouped[toronto_df_grouped["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood


In [40]:
# print the number of rows of the cleaned dataframe
toronto_df_grouped.shape

(103, 3)

In [41]:
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B\n,Scarborough\n,"Malvern, Rouge"
1,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
2,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
3,M1G\n,Scarborough\n,Woburn
4,M1H\n,Scarborough\n,Cedarbrae


In [42]:
toronto_df_grouped

,PostalCode,Borough,Neighborhood
0,M1B\n,Scarborough\n,"Malvern, Rouge"
1,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
2,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
3,M1G\n,Scarborough\n,Woburn
4,M1H\n,Scarborough\n,Cedarbrae
...,...,...,...
98,M9N\n,York\n,Weston
99,M9P\n,Etobicoke\n,Westmount
100,M9R\n,Etobicoke\n,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V\n,Etobicoke\n,"South Steeles, Silverstone, Humbergate, Jamest..."


In [43]:
toronto_df_grouped.to_csv("Week301",index = False)

In [44]:
!pip install geocoder

In [45]:
import pandas as pd
import numpy as np
import geocoder
print("Imported!")

Imported!


In [46]:
df = pd.read_csv('Week301')
df.head()



,PostalCode,Borough,Neighborhood
0,M1B\n,Scarborough\n,"Malvern, Rouge"
1,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
2,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
3,M1G\n,Scarborough\n,Woburn
4,M1H\n,Scarborough\n,Cedarbrae


In [47]:
print(df.shape)

(103, 3)


In [48]:
df.describe()

,PostalCode,Borough,Neighborhood
count,103,103,103
unique,103,10,99
top,M8Z\n,North York\n,Downsview
freq,1,24,4


In [49]:

def get_latilong(postal_code):
    lati_long_coords = None
    while(lati_long_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lati_long_coords = g.latlng
    return lati_long_coords
    
get_latilong('M4G')


[43.70909000000006, -79.36409999999995]

In [51]:
# Retrieving Postal Code Co-ordinates
postal_codes = df['PostalCode']    
coords = [ get_latilong(postal_code) for postal_code in postal_codes.tolist() ]

In [52]:
#lets add the columns lattitude and longitude in our dataframe
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [55]:

df[df.PostalCode == 'M4G']

,PostalCode,Borough,Neighborhood,Latitude,Longitude


In [56]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B\n,Scarborough\n,"Malvern, Rouge",43.81153,-79.19552
1,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek",43.78564,-79.15871
2,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill",43.76575,-79.17520
3,M1G\n,Scarborough\n,Woburn,43.76820,-79.21761
4,M1H\n,Scarborough\n,Cedarbrae,43.76969,-79.23944


In [57]:
df.head(15)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B\n,Scarborough\n,"Malvern, Rouge",43.81153,-79.19552
1,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek",43.78564,-79.15871
2,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill",43.76575,-79.17520
3,M1G\n,Scarborough\n,Woburn,43.76820,-79.21761
4,M1H\n,Scarborough\n,Cedarbrae,43.76969,-79.23944
5,M1J\n,Scarborough\n,Scarborough Village,43.74309,-79.23526
6,M1K\n,Scarborough\n,"Kennedy Park, Ionview, East Birchmount Park",43.72861,-79.26367
7,M1L\n,Scarborough\n,"Golden Mile, Clairlea, Oakridge",43.71406,-79.28412
8,M1M\n,Scarborough\n,"Cliffside, Cliffcrest, Scarborough Village West",43.72360,-79.23496
9,M1N\n,Scarborough\n,"Birch Cliff, Cliffside West",43.69539,-79.26194


In [58]:
df.to_csv("Week302",index = False)